# WebScraping API da Betfair

### Instalando a biblioteca betfairlightweight

In [1]:
pip install betfairlightweight

### Importando as Bibliotecas

In [6]:
import datetime
import pandas as pd
import pytz
import betfairlightweight

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Fazendo login na API da Betfair

In [45]:
dias = 1

username = "EMAIL"
pw = "SENHA"
app_key = "APP_KEY"

# Iniciando a conexão com a API da Betfair
trading = betfairlightweight.APIClient(username, pw, app_key=app_key, cert_files=('./Betfair/CERT.crt','./Betfair/CERT.pem'))
trading.login()

<LoginResource>

### Criando o Filtro de Mercado

In [46]:
filtros_mercado = betfairlightweight.filters.market_filter(
    event_type_ids=['1'], 
    market_start_time={
        'to': (datetime.datetime.utcnow() + datetime.timedelta(days=dias)).strftime("%Y-%m-%dT%TZ")
    }
)

### Listando Eventos e Competições

In [47]:
eventos_futebol = trading.betting.list_events(filter=filtros_mercado)
competicoes_futebol = trading.betting.list_competitions(filter=filtros_mercado)

### Criando um Dicionário para Competições

In [48]:
competitions = {event_object.competition.name: event_object.competition.id 
                for event_object in competicoes_futebol}

### Criando e Organizando o DataFrame

In [49]:
df = pd.DataFrame(list(competitions.items()), columns=['League', 'Id']).sort_values('League').reset_index(drop=True)
df

,League,Id
0,AFC Champions League,39218
1,AFC Cup,12012789
2,Algerian Cup,4215067
3,Argentinian Copa de la Liga Profesional,12225162
4,Argentinian Primera B Nacional,803237
5,Argentinian Primera C,7511144
6,Australian A-League Men,12117172
7,Austrian Bundesliga,10479956
8,Bahraini Premier,12206014
9,Belgian First Division A,89979


### Obter IDs das Competições do DataFrame

In [50]:
competition_ids = df['Id'].to_list()

### Criando Listas Vazias para Armazenar Informações

In [51]:
eventos_list = []
mercado_evento_id = []
nomes_competicoes = []

In [52]:
competition_ids = ['12225162'] 

### Processar Cada Competição

In [53]:
for id_competicao in competition_ids:
    try:
        competicoes = trading.betting.list_competitions(
            filter=betfairlightweight.filters.market_filter(competition_ids=[id_competicao])
        )
        nome_competicao = competicoes[0].competition.name if competicoes else 'Competição não encontrada'
        nomes_competicoes.append(nome_competicao)
    except Exception as e:
        print(f"Erro ao processar competição {id_competicao}: {e}")
        continue

### Configurando Datas e Horários

In [54]:
fuso_horario = 'America/Sao_Paulo' 
agora = datetime.datetime.now(pytz.timezone(fuso_horario)) 
data_atual = agora.date() 
data_futura = data_atual + datetime.timedelta(days=dias)

inicio_do_dia = datetime.datetime.combine(data_atual, datetime.time.min).astimezone(pytz.timezone(fuso_horario))
fim_do_dia = datetime.datetime.combine(data_futura, datetime.time.max).astimezone(pytz.timezone(fuso_horario))

### Definindo os Mercados Desejados

In [55]:
mercados_desejados = [
    "Match Odds", "First Half Goals 0.5", "First Half Goals 1.5", "First Half Goals 2.5",
    "Over/Under 0.5 Goals", "Over/Under 1.5 Goals", "Over/Under 2.5 Goals", "Over/Under 3.5 Goals",
    "Over/Under 4.5 Goals", "Over/Under 5.5 Goals", "Over/Under 6.5 Goals", "Over/Under 7.5 Goals",
    "Over/Under 8.5 Goals", "Both teams to Score?", "Correct Score"
    ]

### Configurando o Filtro de Preços

In [56]:
price_filter = betfairlightweight.filters.price_projection(
    price_data=['EX_BEST_OFFERS'],
    ex_best_offers_overrides=betfairlightweight.filters.ex_best_offers_overrides(best_prices_depth=1)
)

### Iteração com a API da Betfair

In [57]:
# Processando Cada Competição
for competition_id in competition_ids:
    nome_competicao = nomes_competicoes[competition_ids.index(competition_id)]
    
    filtros_mercado = betfairlightweight.filters.market_filter(
        event_type_ids=['1'], 
        market_start_time={'from': inicio_do_dia.strftime('%Y-%m-%dT%H:%M:%SZ'), 'to': fim_do_dia.strftime('%Y-%m-%dT%H:%M:%SZ')},
        competition_ids=[competition_id]
    )
    
    # Obtendo e Filtrando Eventos por Competição
    try:
        eventos_futebol = trading.betting.list_events(filter=filtros_mercado)
    except Exception as e:
        print(f"Erro ao listar eventos para competição {competition_id}: {e}")
        continue
    
    # Processando Cada Evento
    for obj_evento in eventos_futebol:
        id_evento = obj_evento.event.id

        filtro_catalogo_mercados = betfairlightweight.filters.market_filter(event_ids=[id_evento])
        
        # Filtrando e Obtendo Catálogos de Mercado
        try:
            catalogos_mercado = trading.betting.list_market_catalogue(
                filter=filtro_catalogo_mercados, max_results='100', sort='FIRST_TO_START', market_projection=['RUNNER_METADATA']
            )
        except Exception as e:
            print(f"Erro ao listar catálogo de mercado para evento {id_evento}: {e}")
            continue

        market_ids = {}
        odds_dict = {}
        selection_ids = {}
        
        # Processando Cada Mercado no Catálogo
        for mercado in catalogos_mercado:
            if mercado.market_name in mercados_desejados:
                market_ids['IDMercado_' + mercado.market_name.replace(" ", "_").replace(".", "")] = mercado.market_id

                try:
                    market_books = trading.betting.list_market_book(market_ids=[mercado.market_id], price_projection=price_filter)
                except Exception as e:
                    print(f"Erro ao listar book de mercado {mercado.market_id}: {e}")
                    continue
                
                # Obtendo e Processando Informações do Mercado
                for idx, runner in enumerate(market_books[0].runners):
                    try:
                        back_price = runner.ex.available_to_back[0].price if runner.ex.available_to_back else 1.01
                        lay_price = runner.ex.available_to_lay[0].price if runner.ex.available_to_lay else 1000
                        
                        if mercado.market_name == "Match Odds":
                            selection_key = None
                            if idx == 0:
                                nome_coluna = "Match_Odds_Home"
                                selection_key = 'ID_Home'
                            elif idx == 1:
                                nome_coluna = "Match_Odds_Away"
                                selection_key = 'ID_Away'
                            elif idx == 2:
                                nome_coluna = "Match_Odds_Draw"
                                selection_key = 'ID_Draw'
                            if selection_key:
                                selection_ids[selection_key] = runner.selection_id
                        else:
                            nome_coluna = f"{mercado.market_name.replace(' ', '_')}_{runner.selection_id}"

                        odds_dict[nome_coluna] = {'Back': back_price, 'Lay': lay_price}
                    except IndexError:
                        odds_dict[nome_coluna] = {'Back': 1.01, 'Lay': 1000}

        nome_evento = obj_evento.event.name
        times = nome_evento.split(" v ")
        if len(times) != 2:
            print(f"Formato inesperado do nome do evento: {nome_evento}")
            continue

        data_hora = obj_evento.event.open_date.replace(tzinfo=datetime.timezone.utc).astimezone(tz=None)
        
        # Criando e Atualizando um Dicionário de Eventos
        evento_dict = {
            'ID_Evento': id_evento,
            'League': nome_competicao,
            'Date': data_hora.strftime('%Y-%m-%d'),
            'Time': data_hora.strftime('%H:%M:%S'),
            'Home': times[0],
            'Away': times[1]
        }

        evento_dict.update(odds_dict)
        evento_dict.update(market_ids)
        evento_dict.update(selection_ids)
        eventos_list.append(evento_dict)

# Criando uma DataFrame com os Eventos
planilha_eventos_futebol = pd.DataFrame(eventos_list)

### Selecionando Colunas do DataFrame

In [58]:
df = planilha_eventos_futebol[['League','Date','Time','Home','Away',
'Match_Odds_Home','Match_Odds_Away','Match_Odds_Draw',
'First_Half_Goals_0.5_5851482','First_Half_Goals_0.5_5851483',
'First_Half_Goals_1.5_1221385','First_Half_Goals_1.5_1221386',
'First_Half_Goals_2.5_47972','First_Half_Goals_2.5_47973',
'Over/Under_0.5_Goals_5851482','Over/Under_0.5_Goals_5851483',
'Over/Under_1.5_Goals_1221385','Over/Under_1.5_Goals_1221386',
'Over/Under_2.5_Goals_47972','Over/Under_2.5_Goals_47973',
'Over/Under_3.5_Goals_1222344','Over/Under_3.5_Goals_1222345',
'Over/Under_4.5_Goals_1222347','Over/Under_4.5_Goals_1222346',
'Over/Under_5.5_Goals_1485567','Over/Under_5.5_Goals_1485568',
'Over/Under_6.5_Goals_2542448','Over/Under_6.5_Goals_2542449',
'Over/Under_7.5_Goals_1485572','Over/Under_7.5_Goals_1485573',
'Over/Under_8.5_Goals_2407528','Over/Under_8.5_Goals_2407529',
'Both_teams_to_Score?_30246','Both_teams_to_Score?_110503',
'Correct_Score_1','Correct_Score_4','Correct_Score_9','Correct_Score_16',
'Correct_Score_2','Correct_Score_3','Correct_Score_8','Correct_Score_15',
'Correct_Score_5','Correct_Score_6','Correct_Score_7','Correct_Score_14',
'Correct_Score_10','Correct_Score_11','Correct_Score_12','Correct_Score_13',
'Correct_Score_9063254','Correct_Score_9063255','Correct_Score_9063256']]

### Renomeando Colunas

In [59]:
df.columns = ['League','Date','Time','Home','Away',
'Odd_H','Odd_A','Odd_D',
'Odd_Under05_HT','Odd_Over05_HT',
'Odd_Under15_HT','Odd_Over15_HT',
'Odd_Under25_HT','Odd_Over25_HT',
'Odd_Under05_FT','Odd_Over05_FT',
'Odd_Under15_FT','Odd_Over15_FT',
'Odd_Under25_FT','Odd_Over25_FT',
'Odd_Under35_FT','Odd_Over35_FT',
'Odd_Under45_FT','Odd_Over45_FT',
'Odd_Under55_FT','Odd_Over55_FT',
'Odd_Under65_FT','Odd_Over65_FT',
'Odd_Under75_FT','Odd_Over75_FT',
'Odd_Under85_FT','Odd_Over85_FT',
'Odd_BTTS_Yes','Odd_BTTS_No',
'Odd_CS_0x0','Odd_CS_0x1','Odd_CS_0x2','Odd_CS_0x3',
'Odd_CS_1x0','Odd_CS_1x1','Odd_CS_1x2','Odd_CS_1x3',
'Odd_CS_2x0','Odd_CS_2x1','Odd_CS_2x2','Odd_CS_2x3',
'Odd_CS_3x0','Odd_CS_3x1','Odd_CS_3x2','Odd_CS_3x3',
'Odd_CS_Goleada_H','Odd_CS_Goleada_A','Odd_CS_Goleada_D']

### Preparando uma Lista de Colunas Específicas

In [60]:
colunas_para_separar = [
    'Odd_H', 'Odd_A', 'Odd_D', 'Odd_Under05_HT', 'Odd_Over05_HT', 'Odd_Under15_HT', 'Odd_Over15_HT', 
    'Odd_Under25_HT', 'Odd_Over25_HT', 'Odd_Under05_FT', 'Odd_Over05_FT', 'Odd_Under15_FT', 'Odd_Over15_FT', 
    'Odd_Under25_FT', 'Odd_Over25_FT', 'Odd_Under35_FT', 'Odd_Over35_FT', 'Odd_Under45_FT', 'Odd_Over45_FT', 
    'Odd_Under55_FT', 'Odd_Over55_FT', 'Odd_Under65_FT', 'Odd_Over65_FT', 'Odd_Under75_FT', 'Odd_Over75_FT', 
    'Odd_Under85_FT', 'Odd_Over85_FT', 'Odd_BTTS_Yes', 'Odd_BTTS_No', 'Odd_CS_0x0', 'Odd_CS_0x1', 
    'Odd_CS_0x2', 'Odd_CS_0x3', 'Odd_CS_1x0', 'Odd_CS_1x1', 'Odd_CS_1x2', 'Odd_CS_1x3', 'Odd_CS_2x0', 
    'Odd_CS_2x1', 'Odd_CS_2x2', 'Odd_CS_2x3', 'Odd_CS_3x0', 'Odd_CS_3x1', 'Odd_CS_3x2', 'Odd_CS_3x3', 
    'Odd_CS_Goleada_H', 'Odd_CS_Goleada_A', 'Odd_CS_Goleada_D'
]

### Criando um Novo DataFrame

In [61]:
jogos_do_dia = pd.DataFrame()

### Iterando Pelas Colunas do DataFrame Original e Processando Colunas Específicas

In [62]:
for coluna in df.columns:
    if coluna in colunas_para_separar:
        jogos_do_dia[coluna + '_Back'] = df[coluna].apply(lambda x: x['Back'] if isinstance(x, dict) and 'Back' in x else None)
        jogos_do_dia[coluna + '_Lay'] = df[coluna].apply(lambda x: x['Lay'] if isinstance(x, dict) and 'Lay' in x else None)
    else:
        jogos_do_dia[coluna] = df[coluna]

In [63]:
jogos_do_dia

,League,Date,Time,Home,Away,Odd_H_Back,Odd_H_Lay,Odd_A_Back,Odd_A_Lay,Odd_D_Back,Odd_D_Lay,Odd_Under05_HT_Back,Odd_Under05_HT_Lay,Odd_Over05_HT_Back,Odd_Over05_HT_Lay,Odd_Under15_HT_Back,Odd_Under15_HT_Lay,Odd_Over15_HT_Back,Odd_Over15_HT_Lay,Odd_Under25_HT_Back,Odd_Under25_HT_Lay,Odd_Over25_HT_Back,Odd_Over25_HT_Lay,Odd_Under05_FT_Back,Odd_Under05_FT_Lay,Odd_Over05_FT_Back,Odd_Over05_FT_Lay,Odd_Under15_FT_Back,Odd_Under15_FT_Lay,Odd_Over15_FT_Back,Odd_Over15_FT_Lay,Odd_Under25_FT_Back,Odd_Under25_FT_Lay,Odd_Over25_FT_Back,Odd_Over25_FT_Lay,Odd_Under35_FT_Back,Odd_Under35_FT_Lay,Odd_Over35_FT_Back,Odd_Over35_FT_Lay,Odd_Under45_FT_Back,Odd_Under45_FT_Lay,Odd_Over45_FT_Back,Odd_Over45_FT_Lay,Odd_Under55_FT_Back,Odd_Under55_FT_Lay,Odd_Over55_FT_Back,Odd_Over55_FT_Lay,Odd_Under65_FT_Back,Odd_Under65_FT_Lay,Odd_Over65_FT_Back,Odd_Over65_FT_Lay,Odd_Under75_FT_Back,Odd_Under75_FT_Lay,Odd_Over75_FT_Back,Odd_Over75_FT_Lay,Odd_Under85_FT_Back,Odd_Under85_FT_Lay,Odd_Over85_FT_Back,Odd_Over85_FT_Lay,Odd_BTTS_Yes_Back,Odd_BTTS_Yes_Lay,Odd_BTTS_No_Back,Odd_BTTS_No_Lay,Odd_CS_0x0_Back,Odd_CS_0x0_Lay,Odd_CS_0x1_Back,Odd_CS_0x1_Lay,Odd_CS_0x2_Back,Odd_CS_0x2_Lay,Odd_CS_0x3_Back,Odd_CS_0x3_Lay,Odd_CS_1x0_Back,Odd_CS_1x0_Lay,Odd_CS_1x1_Back,Odd_CS_1x1_Lay,Odd_CS_1x2_Back,Odd_CS_1x2_Lay,Odd_CS_1x3_Back,Odd_CS_1x3_Lay,Odd_CS_2x0_Back,Odd_CS_2x0_Lay,Odd_CS_2x1_Back,Odd_CS_2x1_Lay,Odd_CS_2x2_Back,Odd_CS_2x2_Lay,Odd_CS_2x3_Back,Odd_CS_2x3_Lay,Odd_CS_3x0_Back,Odd_CS_3x0_Lay,Odd_CS_3x1_Back,Odd_CS_3x1_Lay,Odd_CS_3x2_Back,Odd_CS_3x2_Lay,Odd_CS_3x3_Back,Odd_CS_3x3_Lay,Odd_CS_Goleada_H_Back,Odd_CS_Goleada_H_Lay,Odd_CS_Goleada_A_Back,Odd_CS_Goleada_A_Lay,Odd_CS_Goleada_D_Back,Odd_CS_Goleada_D_Lay
0,Argentinian Copa de la Liga Profesional,2024-02-18,17:00:00,Velez Sarsfield,Huracan,2.04,2.06,4.70,4.90,3.20,3.30,2.60,2.70,1.59,1.63,1.31,1.34,3.95,4.20,1.07,1.08,13.00,16.5,8.00,9.6,1.12,1.14,2.74,2.88,1.53,1.57,1.56,1.60,2.66,2.76,1.20,1.22,5.50,6.0,1.06,1.08,13.0,18.0,1.01,1.03,34.00,110.0,1.01,1.01,50.00,970.0,1.01,1.01,110.00,970.0,1.01,1.01,100.0,1000.0,2.20,2.36,1.73,1.83,8.0,8.8,11.00,12.5,25.0,32.0,44.0,130.0,5.9,6.6,6.6,7.4,18.50,21.0,38.00,85.0,9.6,11.0,10.5,11.5,23.0,27.0,70.00,120.0,22.0,24.0,25.00,28.0,42.00,980.0,15.50,220.0,20.00,32.0,17.00,140.0,16.00,1000.0
1,Argentinian Copa de la Liga Profesional,2024-02-19,19:15:00,Estudiantes,Newells,2.24,2.36,4.00,4.30,2.92,3.20,1.01,1000.00,1.01,1000.00,1.25,1.35,3.90,5.00,1.05,1.08,13.50,18.5,2.50,970.0,1.07,1.67,2.50,2.70,1.59,1.67,1.50,1.54,2.84,2.98,1.15,1.53,2.84,8.2,1.05,1.12,10.0,26.0,1.02,1.11,10.00,110.0,1.01,1.11,10.00,1000.0,1.01,1.02,10.00,1000.0,1.01,1.01,10.5,1000.0,2.26,2.44,1.70,1.80,6.4,8.4,9.00,11.5,21.0,980.0,8.4,980.0,6.4,7.4,6.4,7.4,14.50,20.0,8.00,980.0,10.0,13.5,9.8,13.5,20.0,980.0,8.40,980.0,8.0,980.0,7.80,980.0,8.00,980.0,8.80,980.0,7.20,980.0,8.40,980.0,9.00,1000.0
2,Argentinian Copa de la Liga Profesional,2024-02-18,17:00:00,Instituto,CA Independiente,2.74,2.82,3.30,3.45,2.92,2.96,2.20,2.26,1.79,1.83,1.22,1.25,5.10,5.60,1.04,1.06,18.00,24.0,5.70,6.2,1.19,1.21,2.20,2.26,1.79,1.84,1.38,1.40,3.50,3.65,1.12,1.14,8.40,9.6,1.03,1.05,22.0,55.0,1.01,1.02,50.00,970.0,1.01,1.01,70.00,970.0,1.01,1.01,100.00,1000.0,1.01,1.01,100.0,1000.0,2.56,2.64,1.60,1.64,6.0,6.4,7.40,8.0,18.0,20.0,32.0,75.0,6.4,6.8,6.4,7.0,15.50,17.5,25.00,70.0,13.5,15.0,14.0,15.0,28.0,34.0,18.50,980.0,25.0,50.0,19.50,55.0,18.00,110.0,20.00,980.0,16.50,75.0,18.00,980.0,19.50,1000.0
3,Argentinian Copa de la Liga Profesional,2024-02-19,17:00:00,Deportivo Riestra,Atl Tucuman,3.35,3.65,2.68,2.80,2.76,2.98,2.14,2.56,1.64,1.87,1.23,1.32,4.10,5.30,1.04,1.07,16.00,22.0,2.26,970.0,1.10,1.79,2.26,2.60,1.63,1.80,1.40,1.52,2.96,3.50,1.12,1.52,2.96,11.0,1.04,1.12,10.0,55.0,1.01,1.11,10.00,110.0,1.01,1.11,10.00,1000.0,1.01,1.02,10.00,1000.0,1.01,1.01,10.5,1000.0,2.28,2.60,1.63,1.78,5.9,7.6,6.00,7.8,12.5,16.5,7.0,980.0,7.4,980.0,5.5,7.2,11.00,980.0,2.98,980.0,18.0,980.0,13.5,980.0,6.4,980.0,2.32,1000.0,4.1,980.0,4.10,1000.0,3.45,1000.

### Processamento do DataFrame para Hoje e Amanhã

In [64]:
def hoje():
    from datetime import date, datetime, timedelta
    dia = date.today() + timedelta(0)
    return dia

def amanha():
    from datetime import date, datetime, timedelta
    dia = date.today() + timedelta(1)
    return dia

def depois_de_amanha():
    from datetime import date, datetime, timedelta
    dia = date.today() + timedelta(2)
    return dia

def drop_reset_index(df):
    df = df.dropna()
    df = df.reset_index(drop=True)
    df.index += 1
    return df

hoje = str(hoje())

flt = jogos_do_dia.Date == hoje
df_hoje = jogos_do_dia[flt]
df_hoje = df_hoje.sort_values(['Time'])
df_hoje = drop_reset_index(df_hoje)
display(df_hoje)

amanha = str(amanha())

flt = jogos_do_dia.Date == amanha
df_amanha = jogos_do_dia[flt]
df_amanha = df_amanha.sort_values(['Time'])
df_amanha = drop_reset_index(df_amanha)
display(df_amanha)
# df_amanha.to_excel(f"Jogos_do_Dia_Betfair_Back_Lay_{amanha}.xlsx",index=False)

depois_de_amanha = str(depois_de_amanha())

flt = jogos_do_dia.Date == depois_de_amanha
df_depois_amanha = jogos_do_dia[flt]
df_depois_amanha = df_depois_amanha.sort_values(['Time'])
df_depois_depois_amanha = drop_reset_index(df_depois_amanha)
display(df_depois_amanha)
# df_amanha.to_excel(f"Jogos_do_Dia_Betfair_Back_Lay_{amanha}.xlsx",index=False)

print("Finalizado!")

,League,Date,Time,Home,Away,Odd_H_Back,Odd_H_Lay,Odd_A_Back,Odd_A_Lay,Odd_D_Back,Odd_D_Lay,Odd_Under05_HT_Back,Odd_Under05_HT_Lay,Odd_Over05_HT_Back,Odd_Over05_HT_Lay,Odd_Under15_HT_Back,Odd_Under15_HT_Lay,Odd_Over15_HT_Back,Odd_Over15_HT_Lay,Odd_Under25_HT_Back,Odd_Under25_HT_Lay,Odd_Over25_HT_Back,Odd_Over25_HT_Lay,Odd_Under05_FT_Back,Odd_Under05_FT_Lay,Odd_Over05_FT_Back,Odd_Over05_FT_Lay,Odd_Under15_FT_Back,Odd_Under15_FT_Lay,Odd_Over15_FT_Back,Odd_Over15_FT_Lay,Odd_Under25_FT_Back,Odd_Under25_FT_Lay,Odd_Over25_FT_Back,Odd_Over25_FT_Lay,Odd_Under35_FT_Back,Odd_Under35_FT_Lay,Odd_Over35_FT_Back,Odd_Over35_FT_Lay,Odd_Under45_FT_Back,Odd_Under45_FT_Lay,Odd_Over45_FT_Back,Odd_Over45_FT_Lay,Odd_Under55_FT_Back,Odd_Under55_FT_Lay,Odd_Over55_FT_Back,Odd_Over55_FT_Lay,Odd_Under65_FT_Back,Odd_Under65_FT_Lay,Odd_Over65_FT_Back,Odd_Over65_FT_Lay,Odd_Under75_FT_Back,Odd_Under75_FT_Lay,Odd_Over75_FT_Back,Odd_Over75_FT_Lay,Odd_Under85_FT_Back,Odd_Under85_FT_Lay,Odd_Over85_FT_Back,Odd_Over85_FT_Lay,Odd_BTTS_Yes_Back,Odd_BTTS_Yes_Lay,Odd_BTTS_No_Back,Odd_BTTS_No_Lay,Odd_CS_0x0_Back,Odd_CS_0x0_Lay,Odd_CS_0x1_Back,Odd_CS_0x1_Lay,Odd_CS_0x2_Back,Odd_CS_0x2_Lay,Odd_CS_0x3_Back,Odd_CS_0x3_Lay,Odd_CS_1x0_Back,Odd_CS_1x0_Lay,Odd_CS_1x1_Back,Odd_CS_1x1_Lay,Odd_CS_1x2_Back,Odd_CS_1x2_Lay,Odd_CS_1x3_Back,Odd_CS_1x3_Lay,Odd_CS_2x0_Back,Odd_CS_2x0_Lay,Odd_CS_2x1_Back,Odd_CS_2x1_Lay,Odd_CS_2x2_Back,Odd_CS_2x2_Lay,Odd_CS_2x3_Back,Odd_CS_2x3_Lay,Odd_CS_3x0_Back,Odd_CS_3x0_Lay,Odd_CS_3x1_Back,Odd_CS_3x1_Lay,Odd_CS_3x2_Back,Odd_CS_3x2_Lay,Odd_CS_3x3_Back,Odd_CS_3x3_Lay,Odd_CS_Goleada_H_Back,Odd_CS_Goleada_H_Lay,Odd_CS_Goleada_A_Back,Odd_CS_Goleada_A_Lay,Odd_CS_Goleada_D_Back,Odd_CS_Goleada_D_Lay
1,Argentinian Copa de la Liga Profesional,2024-02-18,17:00:00,Velez Sarsfield,Huracan,2.04,2.06,4.7,4.90,3.20,3.30,2.60,2.70,1.59,1.63,1.31,1.34,3.95,4.20,1.07,1.08,13.0,16.5,8.00,9.6,1.12,1.14,2.74,2.88,1.53,1.57,1.56,1.60,2.66,2.76,1.20,1.22,5.50,6.0,1.06,1.08,13.0,18.0,1.01,1.03,34.00,110.0,1.01,1.01,50.00,970.0,1.01,1.01,110.0,970.0,1.01,1.01,100.0,1000.0,2.20,2.36,1.73,1.83,8.0,8.8,11.0,12.5,25.0,32.0,44.0,130.0,5.9,6.6,6.6,7.4,18.5,21.0,38.0,85.0,9.6,11.0,10.5,11.5,23.0,27.0,70.0,120.0,22.0,24.0,25.0,28.0,42.0,980.0,15.5,220.0,20.0,32.0,17.0,140.0,16.0,1000.0
2,Argentinian Copa de la Liga Profesional,2024-02-18,17:00:00,Instituto,CA Independiente,2.74,2.82,3.3,3.45,2.92,2.96,2.20,2.26,1.79,1.83,1.22,1.25,5.10,5.60,1.04,1.06,18.0,24.0,5.70,6.2,1.19,1.21,2.20,2.26,1.79,1.84,1.38,1.40,3.50,3.65,1.12,1.14,8.40,9.6,1.03,1.05,22.0,55.0,1.01,1.02,50.00,970.0,1.01,1.01,70.00,970.0,1.01,1.01,100.0,1000.0,1.01,1.01,100.0,1000.0,2.56,2.64,1.60,1.64,6.0,6.4,7.4,8.0,18.0,20.0,32.0,75.0,6.4,6.8,6.4,7.0,15.5,17.5,25.0,70.0,13.5,15.0,14.0,15.0,28.0,34.0,18.5,980.0,25.0,50.0,19.5,55.0,18.0,110.0,20.0,980.0,16.5,75.0,18.0,980.0,19.5,1000.0
3,Argentinian Copa de la Liga Profesional,2024-02-18,19:15:00,River Plate,Banfield,1.49,1.52,9.2,10.50,4.20,4.50,2.94,3.05,1.49,1.52,1.39,1.43,3.35,3.55,1.09,1.11,10.0,12.0,11.00,13.5,1.08,1.10,3.30,3.45,1.41,1.44,1.77,1.82,2.22,2.30,1.28,1.31,4.30,4.6,1.10,1.12,9.4,11.0,1.03,1.05,23.00,36.0,1.01,1.02,50.00,1000.0,1.01,1.01,60.0,1000.0,1.01,1.01,100.0,1000.0,2.40,2.64,1.61,1.71,10.5,11.5,21.0,24.0,14.0,80.0,16.0,1000.0,5.8,6.2,9.2,10.5,30.0,38.0,14.5,210.0,6.8,7.8,9.6,10.5,28.0,36.0,14.5,210.0,11.5,13.5,17.0,19.0,26.0,980.0,14.5,980.0,6.4,10.5,17.5,980.0,17.0,1000.0
4,Argentinian Copa de la Liga Profesional,2024-02-18,21:30:00,Lanus,Boca Juniors,3.25,3.45,2.7,2.80,2.88,2.98,2.20,2.40,1.72,1.84,1.24,1.33,4.00,5.20,1.05,1.07,15.0,25.0,2.32,7.0,1.17,1.20,2.32,2.48,1.68,1.76,1.43,1.48,3.05,3.35,1.15,1.48,3.05,7.6,1.05,1.12,10.0,36.0,1.01,1.12,3.05,980.0,1.01,1.12,3.05,1000.0,1.01,1.11,10.0,1000.0,1.01,1.10,10.0,1000.0,2.34,2.52,1.66,1.75,6.4,7.0,6.8,7.4,14.0,15.5,40.0,48.0,7.8,10.5,6.4,6.8,13.0,14.5,40.0,980.0,17.5,20.0,14.5,980.0,25.0,29.0,12.5,980.0,36.0,70.0,30.0,65.0,12.0,980.0,12.5,270.0,11.5,95.0,11.0,980.0,7.4,1000.0


,League,Date,Time,Home,Away,Odd_H_Back,Odd_H_Lay,Odd_A_Back,Odd_A_Lay,Odd_D_Back,Odd_D_Lay,Odd_Under05_HT_Back,Odd_Under05_HT_Lay,Odd_Over05_HT_Back,Odd_Over05_HT_Lay,Odd_Under15_HT_Back,Odd_Under15_HT_Lay,Odd_Over15_HT_Back,Odd_Over15_HT_Lay,Odd_Under25_HT_Back,Odd_Under25_HT_Lay,Odd_Over25_HT_Back,Odd_Over25_HT_Lay,Odd_Under05_FT_Back,Odd_Under05_FT_Lay,Odd_Over05_FT_Back,Odd_Over05_FT_Lay,Odd_Under15_FT_Back,Odd_Under15_FT_Lay,Odd_Over15_FT_Back,Odd_Over15_FT_Lay,Odd_Under25_FT_Back,Odd_Under25_FT_Lay,Odd_Over25_FT_Back,Odd_Over25_FT_Lay,Odd_Under35_FT_Back,Odd_Under35_FT_Lay,Odd_Over35_FT_Back,Odd_Over35_FT_Lay,Odd_Under45_FT_Back,Odd_Under45_FT_Lay,Odd_Over45_FT_Back,Odd_Over45_FT_Lay,Odd_Under55_FT_Back,Odd_Under55_FT_Lay,Odd_Over55_FT_Back,Odd_Over55_FT_Lay,Odd_Under65_FT_Back,Odd_Under65_FT_Lay,Odd_Over65_FT_Back,Odd_Over65_FT_Lay,Odd_Under75_FT_Back,Odd_Under75_FT_Lay,Odd_Over75_FT_Back,Odd_Over75_FT_Lay,Odd_Under85_FT_Back,Odd_Under85_FT_Lay,Odd_Over85_FT_Back,Odd_Over85_FT_Lay,Odd_BTTS_Yes_Back,Odd_BTTS_Yes_Lay,Odd_BTTS_No_Back,Odd_BTTS_No_Lay,Odd_CS_0x0_Back,Odd_CS_0x0_Lay,Odd_CS_0x1_Back,Odd_CS_0x1_Lay,Odd_CS_0x2_Back,Odd_CS_0x2_Lay,Odd_CS_0x3_Back,Odd_CS_0x3_Lay,Odd_CS_1x0_Back,Odd_CS_1x0_Lay,Odd_CS_1x1_Back,Odd_CS_1x1_Lay,Odd_CS_1x2_Back,Odd_CS_1x2_Lay,Odd_CS_1x3_Back,Odd_CS_1x3_Lay,Odd_CS_2x0_Back,Odd_CS_2x0_Lay,Odd_CS_2x1_Back,Odd_CS_2x1_Lay,Odd_CS_2x2_Back,Odd_CS_2x2_Lay,Odd_CS_2x3_Back,Odd_CS_2x3_Lay,Odd_CS_3x0_Back,Odd_CS_3x0_Lay,Odd_CS_3x1_Back,Odd_CS_3x1_Lay,Odd_CS_3x2_Back,Odd_CS_3x2_Lay,Odd_CS_3x3_Back,Odd_CS_3x3_Lay,Odd_CS_Goleada_H_Back,Odd_CS_Goleada_H_Lay,Odd_CS_Goleada_A_Back,Odd_CS_Goleada_A_Lay,Odd_CS_Goleada_D_Back,Odd_CS_Goleada_D_Lay
1,Argentinian Copa de la Liga Profesional,2024-02-19,17:00:00,Deportivo Riestra,Atl Tucuman,3.35,3.65,2.68,2.80,2.76,2.98,2.14,2.56,1.64,1.87,1.23,1.32,4.10,5.3,1.04,1.07,16.00,22.0,2.26,970.0,1.10,1.79,2.26,2.60,1.63,1.80,1.40,1.52,2.96,3.50,1.12,1.52,2.96,11.0,1.04,1.12,10.0,55.0,1.01,1.11,10.00,110.0,1.01,1.11,10.00,1000.0,1.01,1.02,10.00,1000.0,1.01,1.01,10.5,1000.0,2.28,2.60,1.63,1.78,5.9,7.6,6.00,7.8,12.5,16.5,7.0,980.0,7.4,980.0,5.5,7.2,11.0,980.0,2.98,980.0,18.0,980.0,13.5,980.0,6.4,980.0,2.32,1000.0,4.1,980.0,4.1,1000.0,3.45,1000.0,2.36,1000.0,2.28,980.0,2.24,980.0,2.22,1000.0
2,Argentinian Copa de la Liga Profesional,2024-02-19,19:15:00,Estudiantes,Newells,2.24,2.36,4.00,4.30,2.92,3.20,1.01,1000.00,1.01,1000.00,1.25,1.35,3.90,5.0,1.05,1.08,13.50,18.5,2.50,970.0,1.07,1.67,2.50,2.70,1.59,1.67,1.50,1.54,2.84,2.98,1.15,1.53,2.84,8.2,1.05,1.12,10.0,26.0,1.02,1.11,10.00,110.0,1.01,1.11,10.00,1000.0,1.01,1.02,10.00,1000.0,1.01,1.01,10.5,1000.0,2.26,2.44,1.70,1.80,6.4,8.4,9.00,11.5,21.0,980.0,8.4,980.0,6.4,7.4,6.4,7.4,14.5,20.0,8.00,980.0,10.0,13.5,9.8,13.5,20.0,980.0,8.40,980.0,8.0,980.0,7.8,980.0,8.00,980.0,8.80,980.0,7.20,980.0,8.40,980.0,9.00,1000.0
3,Argentinian Copa de la Liga Profesional,2024-02-19,19:15:00,Barracas Central,Talleres,3.50,4.20,2.24,2.38,3.00,3.45,2.06,2.82,1.33,1.75,1.28,1.39,3.60,4.6,1.06,1.10,12.00,16.0,2.64,960.0,1.07,1.61,2.64,3.05,1.49,1.61,1.54,1.69,2.44,2.84,1.17,1.69,2.44,7.8,1.05,1.12,10.0,36.0,1.01,1.12,2.44,110.0,1.01,1.12,10.00,1000.0,1.01,1.11,10.00,1000.0,1.01,1.11,10.0,1000.0,2.10,2.28,1.79,1.91,7.6,9.6,6.80,8.4,5.1,980.0,5.7,980.0,4.5,980.0,6.6,980.0,4.9,13.5,5.70,980.0,5.6,980.0,5.1,19.0,5.5,980.0,6.20,980.0,4.7,980.0,6.2,980.0,4.10,980.0,4.20,980.0,2.72,980.0,4.30,980.0,2.80,1000.0
4,Argentinian Copa de la Liga Profesional,2024-02-19,21:30:00,Union Santa Fe,CA Platense,2.22,2.38,4.30,5.20,2.82,3.05,2.00,2.72,1.58,2.00,1.20,1.34,3.95,6.0,1.04,1.08,15.00,25.0,2.18,6.0,1.20,1.85,2.18,2.48,1.68,1.85,1.38,1.56,2.80,3.60,1.11,1.56,2.80,11.0,1.04,1.12,10.0,55.0,1.01,1.11,10.00,110.0,1.01,1.11,10.00,1000.0,1.01,1.11,10.00,1000.0,1.01,1.02,10.5,1000.0,2.44,2.64,1.61,1.70,6.0,8.0,3.85,12.5,4.9,38.0,4.4,980.0,5.6,7.4,3.4,8.4,15.0,26.0,4.40,980.0,3.8,15.0,4.2,15.5,5.0,40.0,4.40,980.0,4.2,980.0,4.4,980.0,4.40,980.0,4.50,980.0,4.20,980

,League,Date,Time,Home,Away,Odd_H_Back,Odd_H_Lay,Odd_A_Back,Odd_A_Lay,Odd_D_Back,Odd_D_Lay,Odd_Under05_HT_Back,Odd_Under05_HT_Lay,Odd_Over05_HT_Back,Odd_Over05_HT_Lay,Odd_Under15_HT_Back,Odd_Under15_HT_Lay,Odd_Over15_HT_Back,Odd_Over15_HT_Lay,Odd_Under25_HT_Back,Odd_Under25_HT_Lay,Odd_Over25_HT_Back,Odd_Over25_HT_Lay,Odd_Under05_FT_Back,Odd_Under05_FT_Lay,Odd_Over05_FT_Back,Odd_Over05_FT_Lay,Odd_Under15_FT_Back,Odd_Under15_FT_Lay,Odd_Over15_FT_Back,Odd_Over15_FT_Lay,Odd_Under25_FT_Back,Odd_Under25_FT_Lay,Odd_Over25_FT_Back,Odd_Over25_FT_Lay,Odd_Under35_FT_Back,Odd_Under35_FT_Lay,Odd_Over35_FT_Back,Odd_Over35_FT_Lay,Odd_Under45_FT_Back,Odd_Under45_FT_Lay,Odd_Over45_FT_Back,Odd_Over45_FT_Lay,Odd_Under55_FT_Back,Odd_Under55_FT_Lay,Odd_Over55_FT_Back,Odd_Over55_FT_Lay,Odd_Under65_FT_Back,Odd_Under65_FT_Lay,Odd_Over65_FT_Back,Odd_Over65_FT_Lay,Odd_Under75_FT_Back,Odd_Under75_FT_Lay,Odd_Over75_FT_Back,Odd_Over75_FT_Lay,Odd_Under85_FT_Back,Odd_Under85_FT_Lay,Odd_Over85_FT_Back,Odd_Over85_FT_Lay,Odd_BTTS_Yes_Back,Odd_BTTS_Yes_Lay,Odd_BTTS_No_Back,Odd_BTTS_No_Lay,Odd_CS_0x0_Back,Odd_CS_0x0_Lay,Odd_CS_0x1_Back,Odd_CS_0x1_Lay,Odd_CS_0x2_Back,Odd_CS_0x2_Lay,Odd_CS_0x3_Back,Odd_CS_0x3_Lay,Odd_CS_1x0_Back,Odd_CS_1x0_Lay,Odd_CS_1x1_Back,Odd_CS_1x1_Lay,Odd_CS_1x2_Back,Odd_CS_1x2_Lay,Odd_CS_1x3_Back,Odd_CS_1x3_Lay,Odd_CS_2x0_Back,Odd_CS_2x0_Lay,Odd_CS_2x1_Back,Odd_CS_2x1_Lay,Odd_CS_2x2_Back,Odd_CS_2x2_Lay,Odd_CS_2x3_Back,Odd_CS_2x3_Lay,Odd_CS_3x0_Back,Odd_CS_3x0_Lay,Odd_CS_3x1_Back,Odd_CS_3x1_Lay,Odd_CS_3x2_Back,Odd_CS_3x2_Lay,Odd_CS_3x3_Back,Odd_CS_3x3_Lay,Odd_CS_Goleada_H_Back,Odd_CS_Goleada_H_Lay,Odd_CS_Goleada_A_Back,Odd_CS_Goleada_A_Lay,Odd_CS_Goleada_D_Back,Odd_CS_Goleada_D_Lay
4,Argentinian Copa de la Liga Profesional,2024-02-20,19:30:00,Argentinos Juniors,Independiente Rivadavia,1.79,1.98,4.6,5.9,3.25,3.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.62,990.0,1.05,2.62,NaN,NaN,NaN,NaN,1.62,1.80,2.24,2.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.16,2.38,1.72,1.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Argentinian Copa de la Liga Profesional,2024-02-20,20:00:00,Defensa y Justicia,Central Cordoba (SdE),1.83,1.88,5.2,6.4,3.30,3.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.56,990.0,1.06,2.80,NaN,NaN,NaN,NaN,1.56,1.82,2.22,2.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.14,2.58,1.64,1.88,8.0,12.0,12.0,18.0,4.1,980.0,4.3,980.0,5.2,7.4,6.6,9.6,3.95,980.0,4.2,980.0,8.0,12.0,8.6,13.0,4.0,980.0,4.3,980.0,4.0,980.0,3.95,980.0,4.1,980.0,4.3,1000.0,3.7,28.0,4.3,980.0,4.3,1000.0


Finalizado!
